### Problem 2:

Use the data in problem1.csv.  
Calculate VaR and ES:  
a. Using a normal distribution with an exponentially weighted variance (lambda=0.97);  
b. Using a MLE fitted T distribution  
c. Using a Historic Simulation  
Compare difference between VaR and ES under different probabilistic distributions. Explain the differences.  
使用 "problem1.csv" 数据，计算VaR和ES:  
a. 使用带有指数加权方差（lambda=0.97）的正态分布；  
b. 使用通过最大似然估计（MLE）拟合的T分布；  
c. 使用历史模拟法。  
比较不同概率分布下VaR和ES的差异，并解释这些差异。

In [35]:
import pandas as pd
import numpy as np
from scipy.stats import t
from scipy.stats import norm
from scipy.stats import multivariate_normal
data = pd.read_csv('/Users/apple/Desktop/FT545 - week5/problem1.csv')
returns = data['x']

*normal distribution with an exponentially weighted variance*

In [36]:
def calculate_normal_var_es(returns, confidence_level=0.95, lambda_value=0.97):
    # 计算指数加权方差
    ewma_variance = returns.ewm(alpha=1 - lambda_value).var().iloc[-1]
    ewma_volatility = np.sqrt(ewma_variance)  # 得到标准差（波动率）

    # 计算 VaR
    z_score = norm.ppf(1 - confidence_level)
    var_normal = abs(z_score * ewma_volatility)  # VaR 表示损失，取绝对值

    # 计算 ES
    es_normal = abs(ewma_volatility * norm.pdf(z_score) / (1 - confidence_level))  # ES 表示损失，取绝对值

    return var_normal, es_normal

# 假设 returns 是收益率数据的序列或数组
var_normal, es_normal = calculate_normal_var_es(returns)

print(f"VaR (Normal Distribution with EWMA, 95%): {var_normal:.4f}")
print(f"ES (Normal Distribution with EWMA, 95%): {es_normal:.4f}")


VaR (Normal Distribution with EWMA, 95%): 0.0910
ES (Normal Distribution with EWMA, 95%): 0.1141


在正态分布假设下，VaR 为 9.10%，ES 为 11.41%，是三种方法中 VaR 值最高的。这是因为正态分布假设数据分布集中，导致高估了波动。EWMA 方法强调近期数据波动，对市场波动变化灵敏，但容易在厚尾和极端事件出现时表现不足。

*MLE fitted T distribution*

In [34]:
# 定义一个函数，使用 T 分布计算 VaR 和 ES
def calculate_t_distribution_var_es(returns, confidence_level=0.95):
    # 使用最大似然估计拟合 T 分布
    df, loc, scale = t.fit(returns)
    
    # 计算 VaR：使用 T 分布的逆累积分布函数
    var_t = -t.ppf(1 - confidence_level, df, loc, scale)
    
    # 计算 ES
    es_t = -t.expect(lambda x: x, args=(df,), loc=loc, scale=scale, lb=-np.inf, ub=var_t) / (1 - confidence_level)
    
    return var_t, es_t

# 假设 returns 是收益率数据的序列或数组
var_t, es_t = calculate_t_distribution_var_es(returns)

print(f"VaR (T Distribution, 95%): {var_t:.4f}")
print(f"ES (T Distribution, 95%): {es_t:.4f}")



VaR (T Distribution, 95%): 0.0765
ES (T Distribution, 95%): 0.1144


T 分布假设下的 VaR 为 7.65%，ES 为 11.44%，VaR 比正态分布低，而 ES 接近。这是因为 T 分布能更好地适应厚尾现象，在极端值出现时能提供更合理的估计。T 分布在尾部比正态分布有更大的概率质量，意味着它对极端事件更敏感，适合捕捉市场的极端波动。

*Historic Simulation*

In [33]:
def calculate_historical_var_es(returns, confidence_level=0.95):
    # 将收益率按从小到大排序
    sorted_returns = np.sort(returns)
    
    # 计算 VaR：找到对应置信水平的分位数
    var_index = int((1 - confidence_level) * len(sorted_returns))
    var_historical = -sorted_returns[var_index]  # 取负值表示损失
    
    # 计算 ES：计算所有低于 VaR 的损失的平均值
    es_historical = -sorted_returns[:var_index].mean()  # 取负值表示损失
    
    return var_historical, es_historical

# 假设 returns 是收益率数据的序列或数组
var_historical, es_historical = calculate_historical_var_es(returns)

print(f"VaR (Historical Simulation, 95%): {var_historical:.4f}")
print(f"ES (Historical Simulation, 95%): {es_historical:.4f}")



VaR (Historical Simulation, 95%): 0.0759
ES (Historical Simulation, 95%): 0.1168


历史模拟法的 VaR 为 7.59%，与 T 分布相近，而 ES 为 11.68%，略高于其他方法。历史模拟法直接依赖过去的真实数据，因此反映了实际历史的极端损失情况。此方法不假设数据分布，直接从历史数据中估计损失，因此可以在过去的极端事件上给出更准确的估计。

差异解释:  
极端事件敏感性：正态分布假设的VaR和ES 较高，但对极端事件的估计可能不足；T 分布通过厚尾特性更能捕捉极端风险，而历史模拟法则直接基于历史数据，能够真实反映过去极端事件的损失。  
风险评估的保守程度：正态分布的估计较保守，适合在平稳市场使用；T 分布和历史模拟法则在波动性较高的市场中更能反映真实的极端损失。  
分布假设的适用性：正态分布适合于平稳市场，而 T 分布适用于厚尾数据，历史模拟法在数据充足的情况下最真实可靠。

### Problem 3:  
Use your repository from #1.  
Using Portfolio.csv and DailyPrices.csv. Calculate arithmetic returns. Assume the expected return on all stocks is 0. This file contains the stock holdings of 3 portfolios. You own each of these portfolios.  
Fit Generalized T models to stocks in portfolios A and B, and fit a normal distributions to stocks in portfolio C.  
Calculate the VaR and ES of each portfolio as well as your total VaR and ES. You will need to use a copula. Compare the results from this to your VaR form Problem 3 from Week 4.  

使用您在问题1中的数据仓库，并使用“Portfolio.csv”和“DailyPrices.csv”文件。计算算术收益率，并假设所有股票的预期收益为0。这些文件包含了3个投资组合的股票持仓情况，您持有这些投资组合。

为投资组合A和B中的股票拟合广义T分布模型，为投资组合C中的股票拟合正态分布。计算每个投资组合的VaR和ES，以及所有投资组合的总VaR和ES。您需要使用Copula方法。将此结果与您在第4周问题3中的VaR进行比较。

*A&B*

In [6]:
# 一：导入数据并计算每日收益率

# 读取投资组合和每日价格数据
portfolio_data = pd.read_csv('/Users/apple/Desktop/FT545 - week5/portfolio.csv')
daily_prices = pd.read_csv('/Users/apple/Desktop/FT545 - week5/DailyPrices.csv')

# 转换日期格式
daily_prices['Date'] = pd.to_datetime(daily_prices['Date'])

# 计算每日算术收益率
daily_returns = daily_prices.set_index('Date').pct_change().dropna()

# 检查计算的每日收益率
print(daily_returns.head())



                 SPY      AAPL      MSFT      AMZN      TSLA     GOOGL  \
Date                                                                     
2023-09-06 -0.006723 -0.035793 -0.002009 -0.013914 -0.017817 -0.009649   
2023-09-07 -0.003070 -0.029249 -0.008922  0.018395 -0.001707  0.005950   
2023-09-08  0.001506  0.003492  0.013216  0.002757 -0.011889  0.008280   
2023-09-11  0.006577  0.006623  0.010979  0.035231  0.100925  0.003960   
2023-09-12 -0.005486 -0.017061 -0.018258 -0.013068 -0.022297 -0.011540   

                GOOG      META      NVDA     BRK-B  ...       PNC      MDLZ  \
Date                                                ...                       
2023-09-06 -0.009802 -0.003265 -0.030550  0.003329  ... -0.017678  0.001869   
2023-09-07  0.006131 -0.001671 -0.017424  0.000359  ... -0.021882  0.004448   
2023-09-08  0.007342 -0.002612 -0.014468  0.003731  ...  0.010020 -0.002572   
2023-09-11  0.003936  0.032462 -0.008646  0.006526  ...  0.009322  0.022200   
2023-09

In [7]:
# 二：提取投资组合A和B的股票列表

# 获取投资组合A和B的股票列表
portfolio_A_stocks = portfolio_data[portfolio_data['Portfolio'] == 'A']['Stock'].tolist()
portfolio_B_stocks = portfolio_data[portfolio_data['Portfolio'] == 'B']['Stock'].tolist()

# 输出以确认股票列表
print("Portfolio A Stocks:", portfolio_A_stocks)
print("Portfolio B Stocks:", portfolio_B_stocks)


Portfolio A Stocks: ['AAPL', 'TSLA', 'JPM', 'HD', 'BAC', 'XOM', 'AVGO', 'PEP', 'TMO', 'CMCSA', 'META', 'ACN', 'INTC', 'PYPL', 'MRK', 'T', 'LOW', 'NEE', 'AMD', 'INTU', 'MS', 'AMAT', 'CVS', 'GS', 'BA', 'SBUX', 'GE', 'ISRG', 'MU', 'NOW', 'ADP', 'PNC', 'ADI', 'SYK', 'TJX']
Portfolio B Stocks: ['MSFT', 'GOOGL', 'NVDA', 'JNJ', 'PG', 'MA', 'DIS', 'ADBE', 'KO', 'NFLX', 'COST', 'WFC', 'WMT', 'LLY', 'NKE', 'LIN', 'UNP', 'UPS', 'MDT', 'ORCL', 'RTX', 'AMGN', 'CAT', 'AMT', 'COP', 'AXP', 'SPGI', 'BKNG', 'ZTS', 'MDLZ', 'GILD', 'GM']


In [8]:
# 三：提取投资组合A和B的股票收益率

# 提取投资组合A和B的股票收益率
returns_A = daily_returns[portfolio_A_stocks]
returns_B = daily_returns[portfolio_B_stocks]

# 检查数据
print("Returns for Portfolio A:")
print(returns_A.head())
print("\nReturns for Portfolio B:")
print(returns_B.head())

Returns for Portfolio A:
                AAPL      TSLA       JPM        HD       BAC       XOM  \
Date                                                                     
2023-09-06 -0.035793 -0.017817 -0.001653 -0.005431 -0.009075  0.008632   
2023-09-07 -0.029249 -0.001707 -0.008554  0.007423 -0.009158 -0.004890   
2023-09-08  0.003492 -0.011889  0.000765  0.000883  0.008176  0.014568   
2023-09-11  0.006623  0.100925  0.004380 -0.001035  0.004231 -0.012542   
2023-09-12 -0.017061 -0.022297  0.013014 -0.006365  0.017205  0.029170   

                AVGO       PEP       TMO     CMCSA  ...      SBUX        GE  \
Date                                                ...                       
2023-09-06  0.000115  0.003273 -0.009797  0.008303  ... -0.009190 -0.001772   
2023-09-07 -0.017472  0.009558 -0.009390 -0.002003  ... -0.008859 -0.007898   
2023-09-08  0.000607 -0.000737 -0.025185  0.004237  ...  0.001893 -0.000626   
2023-09-11  0.001854  0.015090 -0.006425  0.005108  ...  0.00

In [13]:
# 四：定义广义T分布的VaR和ES计算函数

# 定义一个函数，基于广义T分布拟合数据并返回VaR和ES
def fit_t_distribution(returns, confidence_level=0.95):
    # 使用最大似然估计拟合T分布
    df, loc, scale = t.fit(returns)
    
    # 计算VaR和ES
    var = -t.ppf(1 - confidence_level, df, loc, scale)
    es = -t.expect(lambda x: x, args=(df,), loc=loc, scale=scale, lb=-np.inf, ub=var) / (1 - confidence_level)
    
    return var, es


In [26]:
# 五：计算投资组合A每只股票的VaR和ES

# 对投资组合A的每只股票拟合T分布，并计算VaR和ES
var_es_A = {stock: fit_t_distribution(returns_A[stock].dropna()) for stock in portfolio_A_stocks}
print("VaR and ES for Portfolio A:", var_es_A)

# 计算A的总Var&ES

# 获取组合A中每只股票的持仓数量
holdings_A = portfolio_data[portfolio_data['Portfolio'] == 'A'].set_index('Stock')['Holding']

# 计算投资组合A的加权总收益率
weighted_returns_A = (returns_A * holdings_A).sum(axis=1) / holdings_A.sum()

# 基于总收益率拟合T分布并计算VaR和ES
VaR_A, ES_A = fit_t_distribution(weighted_returns_A.dropna())

print(f"Total VaR for Portfolio A (95%): {VaR_A:.4f}")
print(f"Total ES for Portfolio A (95%): {ES_A:.4f}")



VaR and ES for Portfolio A: {'AAPL': (np.float64(0.020825934309671175), np.float64(0.019268493169752918)), 'TSLA': (np.float64(0.05408340858800977), np.float64(0.09493998286067973)), 'JPM': (np.float64(0.014683213592155473), np.float64(-0.005054003132215513)), 'HD': (np.float64(0.020456302420638933), np.float64(0.020852679516762417)), 'BAC': (np.float64(0.02233292655198873), np.float64(0.019347379372228447)), 'XOM': (np.float64(0.020562900277373286), np.float64(0.022979253738536883)), 'AVGO': (np.float64(0.039564089540108247), np.float64(0.03778956127899833)), 'PEP': (np.float64(0.0161458612163748), np.float64(0.0216341879488148)), 'TMO': (np.float64(0.02022105505873821), np.float64(0.02222719684403385)), 'CMCSA': (np.float64(0.021686996912826738), np.float64(0.03171625454381218)), 'META': (np.float64(0.029936252213129316), np.float64(0.02081557506280513)), 'ACN': (np.float64(0.0199637475161646), np.float64(0.018962582789905296)), 'INTC': (np.float64(0.04063695022397263), np.float64(0.

组合 A 的 VaR 和 ES 值较低，表明该组合在 95% 的置信水平下有较小的损失风险。这可能是由于组合 A 的波动性较低，或其成分股的厚尾风险不高。

In [27]:
# 六：计算投资组合B每只股票的VaR和ES

# 对投资组合B的每只股票拟合T分布，并计算VaR和ES
var_es_B = {stock: fit_t_distribution(returns_B[stock].dropna()) for stock in portfolio_B_stocks}
print("VaR and ES for Portfolio B:", var_es_B)

# 计算B的总Var&ES

# 获取组合B中每只股票的持仓数量
holdings_B = portfolio_data[portfolio_data['Portfolio'] == 'B'].set_index('Stock')['Holding']

# 计算投资组合B的加权总收益率
weighted_returns_B = (returns_B * holdings_B).sum(axis=1) / holdings_B.sum()

# 基于总收益率拟合T分布并计算VaR和ES
VaR_B, ES_B = fit_t_distribution(weighted_returns_B.dropna())

print(f"Total VaR for Portfolio B (95%): {VaR_B:.4f}")
print(f"Total ES for Portfolio B (95%): {ES_B:.4f}")

VaR and ES for Portfolio B: {'MSFT': (np.float64(0.019008035310028624), np.float64(0.010387836402863925)), 'GOOGL': (np.float64(0.02317195155140554), np.float64(0.010903306119250784)), 'NVDA': (np.float64(0.04584259655556917), np.float64(0.022491019895041344)), 'JNJ': (np.float64(0.014889890726177636), np.float64(0.01671556649063981)), 'PG': (np.float64(0.013573086165002853), np.float64(0.01112054516551797)), 'MA': (np.float64(0.01817717413087261), np.float64(0.03334745158610575)), 'DIS': (np.float64(0.022977426921169646), np.float64(0.026798519515062732)), 'ADBE': (np.float64(0.02840920894716193), np.float64(0.03565469238180311)), 'KO': (np.float64(0.012020282234804043), np.float64(0.006385414649783003)), 'NFLX': (np.float64(0.026899970097375363), np.float64(0.028161801160198608)), 'COST': (np.float64(0.015666996873818312), np.float64(-0.004223780083016853)), 'WFC': (np.float64(0.02183887940046434), np.float64(0.012600330893231601)), 'WMT': (np.float64(0.013988632020868419), np.float6

组合 B 的 VaR 和 ES 是三个组合中最低的，尤其是 ES 值仅为 0.0010。这说明组合 B 的成分股波动性最小，或具有极低的极端风险事件发生概率。

*C*

In [16]:
# 七：提取投资组合C的股票列表和收益率

# 获取投资组合C的股票列表
portfolio_C_stocks = portfolio_data[portfolio_data['Portfolio'] == 'C']['Stock'].tolist()

# 提取投资组合C的股票收益率
returns_C = daily_returns[portfolio_C_stocks]

# 检查数据
print("Returns for Portfolio C:")
print(returns_C.head())

Returns for Portfolio C:
                AMZN      GOOG     BRK-B       UNH         V       PFE  \
Date                                                                     
2023-09-06 -0.013914 -0.009802  0.003329 -0.008049  0.003342 -0.028547   
2023-09-07  0.018395  0.006131  0.000359  0.016501  0.003981 -0.002619   
2023-09-08  0.002757  0.007342  0.003731 -0.004473  0.000607 -0.000875   
2023-09-11  0.035231  0.003936  0.006526 -0.002891 -0.000283 -0.009051   
2023-09-12 -0.013068 -0.012124  0.006183  0.001085  0.000324  0.006188   

                CSCO       CVX      ABBV       ABT  ...       IBM       PLD  \
Date                                                ...                       
2023-09-06 -0.001746  0.001021 -0.003354  0.006741  ... -0.000473 -0.005490   
2023-09-07 -0.006473  0.000060  0.022663 -0.008763  ... -0.003647  0.015901   
2023-09-08 -0.002113  0.003420  0.000739  0.000596  ...  0.001084 -0.006731   
2023-09-11  0.003000 -0.020633  0.000134  0.017075  ...  0.00

In [17]:
# 八：定义正态分布的VaR和ES计算函数

# 定义一个函数，基于正态分布拟合数据并返回VaR和ES
def fit_normal_distribution(returns, confidence_level=0.95):
    # 计算正态分布的均值和标准差
    mu, sigma = returns.mean(), returns.std()
    
    # 计算VaR和ES
    var = -norm.ppf(1 - confidence_level, mu, sigma)
    es = -mu + sigma * norm.pdf(norm.ppf(confidence_level)) / (1 - confidence_level)
    
    return var, es

In [28]:
# 九：计算投资组合C每只股票的VaR和ES

# 对投资组合C的每只股票拟合正态分布，并计算VaR和ES
var_es_C = {stock: fit_normal_distribution(returns_C[stock].dropna()) for stock in portfolio_C_stocks}
print("VaR and ES for Portfolio C:", var_es_C)

# 计算C的总VaR和ES

# 获取组合C中每只股票的持仓数量
holdings_C = portfolio_data[portfolio_data['Portfolio'] == 'C'].set_index('Stock')['Holding']

# 计算投资组合C的加权总收益率
weighted_returns_C = (returns_C * holdings_C).sum(axis=1) / holdings_C.sum()

# 基于总收益率拟合正态分布并计算VaR和ES
VaR_C, ES_C = fit_normal_distribution(weighted_returns_C.dropna())

print(f"Total VaR for Portfolio C (95%): {VaR_C:.4f}")
print(f"Total ES for Portfolio C (95%): {ES_C:.4f}")

VaR and ES for Portfolio C: {'AMZN': (np.float64(0.0284309571153024), np.float64(0.035962811438063666)), 'GOOG': (np.float64(0.027709521504481978), np.float64(0.034981206137133444)), 'BRK-B': (np.float64(0.012280920168106734), np.float64(0.01569283782259844)), 'UNH': (np.float64(0.022054155916520504), np.float64(0.027906576964900762)), 'V': (np.float64(0.015119251666542504), np.float64(0.0191012294454893)), 'PFE': (np.float64(0.026339974213111735), np.float64(0.03291875376650724)), 'CSCO': (np.float64(0.021205211894992138), np.float64(0.02651785885647531)), 'CVX': (np.float64(0.021510529676751804), np.float64(0.0269194810185493)), 'ABBV': (np.float64(0.018128523087130848), np.float64(0.02309211917129414)), 'ABT': (np.float64(0.018697103684653427), np.float64(0.02360361219416859)), 'CRM': (np.float64(0.034615692711795036), np.float64(0.04362228755491514)), 'VZ': (np.float64(0.0216787993125612), np.float64(0.027483228418605926)), 'QCOM': (np.float64(0.03597753765331338), np.float64(0.045

组合 C 的 VaR 和 ES 都高于组合 A 和 B。尤其是 ES 值为 0.0171，远高于组合 A 和 B。这可能因为组合 C 使用了正态分布假设，相对于广义 T 分布，其在极端事件下的风险估计偏低。

*all*

In [21]:
# 计算整体组合的VaR和ES（Copula）
# 一：转换收益率为均匀分布变量: 使用广义T分布（组合A和B）和正态分布（组合C）的CDF，将收益率转换到[0, 1]上

# A (T分布)
uniform_returns_A = {stock: t.cdf(returns_A[stock].dropna(), *t.fit(returns_A[stock].dropna())) for stock in portfolio_A_stocks}

# B (T分布)
uniform_returns_B = {stock: t.cdf(returns_B[stock].dropna(), *t.fit(returns_B[stock].dropna())) for stock in portfolio_B_stocks}

# C (正态分布)
uniform_returns_C = {stock: norm.cdf(returns_C[stock].dropna(), returns_C[stock].mean(), returns_C[stock].std()) for stock in portfolio_C_stocks}

In [22]:
# 二: 计算所有股票的Spearman相关矩阵，用于高斯Copula

# 将所有统一的返回结果合并到一个df中进行相关性计算
uniform_data = pd.DataFrame({**uniform_returns_A, **uniform_returns_B, **uniform_returns_C})

# 计算Spearman相关矩阵
spearman_corr = uniform_data.corr(method='spearman')
print("Spearman Correlation Matrix:\n", spearman_corr)

Spearman Correlation Matrix:
           AAPL      TSLA       JPM        HD       BAC       XOM      AVGO  \
AAPL  1.000000  0.411324  0.139604  0.236985  0.128881 -0.039452  0.404532   
TSLA  0.411324  1.000000  0.223339  0.314044  0.224567  0.003561  0.393592   
JPM   0.139604  0.223339  1.000000  0.373891  0.694100  0.296555  0.152833   
HD    0.236985  0.314044  0.373891  1.000000  0.391398  0.158459  0.314658   
BAC   0.128881  0.224567  0.694100  0.391398  1.000000  0.319939  0.111810   
...        ...       ...       ...       ...       ...       ...       ...   
F     0.157899  0.354919  0.416043  0.429747  0.527421  0.257078  0.245880   
LRCX  0.366937  0.405984  0.188625  0.379761  0.217526 -0.025930  0.739533   
MO    0.145543  0.103643  0.347630  0.282476  0.359904  0.267252 -0.113828   
LMT   0.015950 -0.080201  0.215747  0.102345  0.138022  0.310543 -0.186383   
TFC   0.167373  0.345002  0.590493  0.500423  0.728305  0.258524  0.195300   

           PEP       TMO     CMCS

In [23]:
# 三：利用Spearman相关矩阵创建高斯Copula模型，并生成模拟的联合分布

# 模拟次数
num_simulations = 10000

# 基于高斯联结公式生成模拟均匀收益
normal_copula = multivariate_normal.rvs(cov=spearman_corr, size=num_simulations)
simulated_uniforms = norm.cdf(normal_copula)

# 转换为df以方便处理
simulated_uniforms_df = pd.DataFrame(simulated_uniforms, columns=uniform_data.columns)


In [24]:
# 四： 通过各股票的逆CDF（Quantile Function），将模拟的均匀变量转换为对应的收益率

simulated_returns = pd.DataFrame()

# A和B的T分布
for stock in portfolio_A_stocks:
    params = t.fit(returns_A[stock].dropna())
    simulated_returns[stock] = t.ppf(simulated_uniforms_df[stock], *params)

for stock in portfolio_B_stocks:
    params = t.fit(returns_B[stock].dropna())
    simulated_returns[stock] = t.ppf(simulated_uniforms_df[stock], *params)

# 正态分布的C
for stock in portfolio_C_stocks:
    mu, sigma = returns_C[stock].mean(), returns_C[stock].std()
    simulated_returns[stock] = norm.ppf(simulated_uniforms_df[stock], mu, sigma)


In [25]:
# 五： 根据模拟的收益率数据，计算整个组合的VaR和ES

# 计算每个模拟的投资组合回报（假设权重相等）
portfolio_returns = simulated_returns.sum(axis=1)

# 在95%置信水平下计算VaR和ES
confidence_level = 0.95
VaR_portfolio = -np.percentile(portfolio_returns, (1 - confidence_level) * 100)
ES_portfolio = -portfolio_returns[portfolio_returns <= -VaR_portfolio].mean()

print(f"Total Portfolio VaR (95%): {VaR_portfolio:.4f}")
print(f"Total Portfolio ES (95%): {ES_portfolio:.4f}")

Total Portfolio VaR (95%): 1.1785
Total Portfolio ES (95%): 1.5792


总组合的 VaR 和 ES 比单独组合高出一个数量级，这说明在多组合持仓时，不同组合间的相关性放大了极端事件下的整体损失。单独组合的 VaR 和 ES 只是其个体风险，但在组合间有相关性时，总 VaR 和 ES 将会远大于各自组合风险的简单相加。这表明风险累积强调了在多组合策略中考虑关联性和极端损失分布的重要性。